## import 导入链的注意事项

导入是将被导入模块中的名称添加到导入模块的命名空间中的过程。

假设当前模块名为 `M`，`from X import Xo` 会将 `Y` 直接添加到当前模块的命名空间：可以访问 `M.Xo`

而 `import X` 会将整个模块 `X` 作为一个对象添加到当前模块的命名空间：`M.X`（包括例如 `M.X.Xo`）等。

例子 1：

```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
from A import value, func  # 从A导入特定内容到B的命名空间

# 模块C (c.py)
import B

# 在C中可以使用:
print(B.value)  # 可以访问，输出100
print(B.func())  # 可以访问，输出"A.func"
```
例子 2：
```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
import A  # A成为B模块命名空间中的一个对象

# 模块C (c.py)
import B

# 在C中:
print(B.A.value)  # 可以访问，输出100
print(B.value)    # 错误! B中没有直接定义value

## 描述符
### 描述符协议

描述符协议规定了：`描述符` 作为某个属主类的 `类属性` 时，属主类使用该描述符时实际发生的行为。

一个类要成为 `描述符`，至少要实现下面三个特殊方法中的一个：

```python
__get__(self, instance, owner)
__set__(self, instance, value)
__delete__(self, instance)
```

其中 `instance` 是属主类实例，`owner` 是属主类本身。

```python
class Descriptor:

    def __init__(self):
        ...

    def __get__(self, instance, owner):
        ...

    def __set__(self, instance, value):
        ...

    def __delete__(self, instance):
        ...

class User:
    descriptor = Descriptor()

user = User()
```

当类似于 `= user.descriptor` 使用时，实际上是触发了 `descriptor.__get__(user, User)`。

`user.descriptor = 值` 实际上触发了 `descriptor.__set__(user, 值)`。

`del user.descriptor` 实际上触发了 `descriptor.__delete__(user)`。

### property 类

`property` 是一个实现了描述符协议的 Python 内置类，其实现：

```python
class Property:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc"

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    # 返回新的 Property 实例
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
```

### @property

使用方法如下

```python
class MyClass:
    def __init__(self):

    # value = property(函数 value(self))
    @property
    def value(self):
        ...

    # value = value.setter(函数 value(self, new_value))
    @value.setter
    def value(self, new_value):
        ...

    # value = value.setter(函数 value(self))
    @value.deleter
    def value(self):
        ...
```

所以相当于创建了三次 property 实例，最后该实例成为 MyClass 的一个类属性 value。

然后 MyClass 或其实例就可以按照 `描述符协议` 来使用 value。

## `__dict__` 和字典类的 `keys()`
### `__setattr__` 和 `__setitem__`
通过 `点表示法` 给实例的属性赋值时，会调用 `__setattr__`，默认情况下，会将该属性的值更新到该实例的 `__dict__`。

实现了 `__setitem__` 方法的类型，其实例使用 `[]` 索引法赋值时，会调用 `__setitem__`。
### `__dict__` 和字典类的 `keys()`

类型和实例都有 `__dict__`，其可以理解为一个字典。
实例的 `__dict__` 存储了实例属性（包括 `__init__` 中 `self.* =`为所有实例绑定的，以及动态绑定的），而类型的 `__dict__` 存储了类属性和方法。

字典 `dict` 有 `keys()`、`values()` 和 `items()` 方法。继承了字典的类自然也继承了这三个方法。
### Config 类的 `_as_attrs_`, `__setattr__`, `__setitem__`
注意到 `Config(PickleableDict, Documented)<——PickleableDict(Pickleable, dict)`，所以 `Config` 是字典类。

`Config` 作为字典类，以及 `_as_attrs_`, `_readonly_`, `_frozen_keys_`, `__setattr__`, `__setitem__` 的实现，构成了：
- `__dict__` 和 `dict` 是两个存储实例属性的结构；
- `_as_attrs_` 是 `false`：点赋值无效，只提供了索引赋值`dict`（且不能是`非强制更新 && _readonly_ || 非强制更新 && _frozen_keys_ && 索引的键在 dict 中` 的情况）；
- `_as_attrs_` 是 `true`：无论点赋值或索引赋值，`非强制更新 && _readonly_ || 非强制更新 && _frozen_keys_ && 索引的键在 dict 中` 决定了是否更新 `__dict__` 和 `dict`；

所以这种设计思想是：
- `self` 具有 `items()` 和 `__dict__` 两个属性存储结构。
- `_as_attrs_` 决定了是否支持点赋值
- 当实施赋值时，`_readonly_` 和 `_frozen_keys_` 决定了是否允许赋值。
- 允许赋值后，`_as_attrs_` 还决定了是否更新 `__dict__`。

## `with open(str, '*') as f:`
等价于
```python
try:
    f = open(str, '*')
    ...
finally:
    if f:
        f.close()
```

## `__new__(cls, *args, **kwargs)`

### 类的方法定义中的 `self` 和 `cls`
```python
class MyClass:
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)

    @classmethod
    def funcClass(cls, *args, **kwargs):...

    def func(self, *args, **kwargs):...

instance = MyClass()
instance.func()
```
当 `instance.func()` 时，实例方法会自动将实例 `instance` 作为第一个参数 `self`。

当 `instance.funcClass()` 或 `MyClass.funcClass()` 时，类方法会识别出类型作为第一个参数 `cls`。

至于为什么一般写为 `self` 和 `cls`，这只是约定俗成的惯例。

### `__new__(cls, *args, **kwargs)` 是静态方法
`__new__` 主要作用是分配内存并创建实例本身，其一般是通知父类的 `__new__` 创建对应类型的实例：
```python
def __new__(cls, *args, **kwargs):
    return super().__new__(cls)
```
如果父类直接继承自 `object` 或者没有重定义 `__new__`，那么最终实际上是 `object` 调用其 `__new__` 创建对应类型的实例。

由于 `__new__(cls, *args, **kwargs)` 不是类方法，所以 `cls` 是手动传入的，其作用在于告诉 `object` 要创建什么类型的实例。
### 类型创建实例的过程
当一个类型创建实例时，例如 `instance = MyClass()`，Python 内部实际执行
```python
instance = MyClass.__new__(MyClass)
if isinstance(instance, MyClass):
    instance.__init__()
```
`instance = MyClass.__new__(MyClass)` 这一行相当于告诉 `object` 创建一个 MyClass 类型的实例，而 `instance.__init__()` 负责初始化。

## Python 类型和实例的内存构成
### 实例
`__dict__` 字典的引用：存储实例在运行时动态添加或在 `__init__` 方法中设置的属性。
- 如果一个类定义了 `__slots__`，并且其中没有包含 `__dict__`，那么该类的实例将不会有 `__dict__` 属性。这种情况下，实例属性（名称包含在 `__slots__` 中）在内存中是独立存储的
- 许多内置类型的实例（如 list, tuple, str, int, set, map 对象，甚至 dict 实例本身）为了效率和设计目的，没有 `__dict__` 属性，属性直接存储在固定内存布局中

`__class__` 引用：指向实例所属的类对象（如 `<class '__main__.A'>`）

`__weakref__`：支持弱引用机制，仅在需要时存在。
### 类型
`__dict__` 字典的引用：存储类属性、方法、描述符等。

`__name__`：类的名称。

`__module__`：类定义所在的模块名。

`__bases__`：类的父类元组

`__doc__`：类的文档字符串。

`__qualname__ `：类的限定名称（包含嵌套类路径）。

`方法表`：存储类的方法（如 `__init__`, `__str__`）。

## 实例的内存理解
`域` 即一段内存空间
- 每个域具有 `域类型`
- 域分为`值域`和`引用域`
    - 值域存储的是实际内容 `域名.字段名 = 值`
    - 引用域存储的是另一个域的地址 `域名.字段名 = 域名`
- 对于一个域 $o$，从其引用域延申到其它域，这样一直延申，可以得到一个分层次的、由多个域构成、多分支有向连接的 `图`
    - 当然可能有其它域的字段引用 `o`，但是这里只考虑从 `o` 出发延申得到的
    - 图上的两个域 $a$ 和 $b$，如果 $a.x=b$，则称有一条 $a\mathop  \to \limits^{a.x} b$ 的`路径`。
        - 如果存在形如 $a \to  \cdots  \to a$ 的路径，则称该图是 `有环图`
    - 如何不重复的遍历图中的所有域：
        ```python
        def Vist(域x, visited):
            if visited.contains(id(x))
                return
            visited[id(x)] = True
            对于 x.引用域中的每个引用i：
                Vist(i, visited)
        ```

## 浅拷贝和深拷贝
### 浅拷贝
将被拷贝实例的域中的内容完整的拷贝到另一段内存空间。所以两段内存空间中的引用域指向相同的实例。
### 深拷贝
深拷贝就是在内存中开辟另一个图，使得值域、对应有向连接和原图相同。

假设有一个函数 `deepcopy(实例x, memo)`，返回深拷贝实例x后的地址，其伪代码如下：
```python
def deepcopy(实例x, memo):
    if memo.contains(id(x))
        return memo[id(x)]
    内存中创建一个和x同类型的实例y
    memo[id(x)] = y
    y.值域 = x.值域
    遍历 x.引用域中的每个引用i：
        y.i = deepcopy(引用i，memo)
    return y
```

## 代码分析
### 函数 `convert_to_dict`
```python
def convert_to_dict(dct: InConfigLikeT, nested: bool = True) -> dict:
    if dct is None:
        dct = {}
    if isinstance(dct, atomic_dict):
        dct = atomic_dict(dct)
    else:
        dct = dict(dct) 
    if not nested:
        return dct
    for k, v in dct.items():
        if isinstance(v, dict):
            dct[k] = convert_to_dict(v, nested=nested)
        else:
            dct[k] = v
    return dct
```

将dict型域dct的图中的dict型（非atomic_dict）域转换为纯dict（nested决定考虑所有域还是只有直接域）。

### 函数 `update_dict`
```python
def update_dict(x: InConfigLikeT,
                y: InConfigLikeT,
                nested: bool = True,
                force: bool = False,
                same_keys: bool = False) -> None:
    if x is None:
        return
    if y is None:
        return
    checks.assert_instance_of(x, dict)
    checks.assert_instance_of(y, dict)
    for k, v in y.items():
        if nested \
                and k in x \
                and isinstance(x[k], dict) \
                and isinstance(v, dict) \
                and not isinstance(v, atomic_dict):
            update_dict(x[k], v, force=force)
        else:
            if same_keys and k not in x:
                continue
            set_dict_item(x, k, v, force=force)
```

首先注意 `x.items()` 和 `y.items()` 都是无环图。

考虑 `x.items()` 和 `y.items()` 路径域类型全为dict、去尾路径字段名相同的域（`nested` 决定考虑所有域还是只有直接域）：
- 两个域中有字段重名 `并且` ( 该字段至少有一个非dict类型 `或者` `y.items()`对应域的该字段为atomoic_dict类型) 
  - 则用 `y.items()` 的该字段的值直接替换 `x.items()` 该字段的值
- 如果 `same_keys` 关闭，且 `y.items()`对应域的某字段名在 `x.items()` 对应域中不存在
  - 用 `y.items()` 对应域的该字段的值在 `x.items()` 对应域中生成该字段

### 函数 `copy_dict`
```python
def copy_dict(dct: InConfigLikeT, copy_mode: str = 'shallow', nested: bool = True) -> OutConfigLikeT:
    if dct is None:
        dct = {}
    checks.assert_instance_of(copy_mode, str)
    copy_mode = copy_mode.lower()
    if copy_mode not in ['shallow', 'hybrid', 'deep']:
        raise ValueError(f"Copy mode '{copy_mode}' not supported")

    if copy_mode == 'deep':
        return deepcopy(dct)
    
    if isinstance(dct, Config):
        return dct.copy(
            copy_mode=copy_mode,
            nested=nested
        )
        
    dct_copy = copy(dct) 
    for k, v in dct_copy.items():
        if nested and isinstance(v, dict):
            _v = copy_dict(v, copy_mode=copy_mode, nested=nested)
        else:
            if copy_mode == 'hybrid':
                _v = copy(v)  
            else:
                _v = v
        set_dict_item(dct_copy, k, _v, force=True)
    return dct_copy
```